In [3]:
import hoomd
import numpy as np

from monk import prep, pair

In [38]:

class CustomAction(hoomd.custom.Action):

    def __init__(self):
        pass

    def act(self, timestep):
        with self._state.cpu_local_snapshot as data:
            pos = data.particles.position._coerce_to_ndarray()
            rtags = data.particles.rtag._coerce_to_ndarray()
            tags = data.particles.tag._coerce_to_ndarray()
            print(f"\n\n\n{timestep}")
            print(pos[24])
            print(rtags[24])
            print(tags[24])

In [27]:
seed = 1000
device = hoomd.device.CPU()
sim = hoomd.Simulation(device, seed=seed)
N = 64
rng = prep.init_rng(seed + 1)
L = prep.len_from_phi(N, 1.2, dim=2)
snap = prep.approx_euclidean_snapshot(N, L, rng, dim=2, ratios=[60, 40], diams=[1.0, 0.88])

print(snap.particles.position[:10])

sim.create_state_from_snapshot(snap)

[[-3.65148372 -3.65148372  0.        ]
 [-2.73861279 -3.65148372  0.        ]
 [-1.82574186 -3.65148372  0.        ]
 [-0.91287093 -3.65148372  0.        ]
 [ 0.         -3.65148372  0.        ]
 [ 0.91287093 -3.65148372  0.        ]
 [ 1.82574186 -3.65148372  0.        ]
 [ 2.73861279 -3.65148372  0.        ]
 [-3.65148372 -2.73861279  0.        ]
 [-2.73861279 -2.73861279  0.        ]]


In [28]:
integrator = hoomd.md.Integrator(dt=0.0025)
tree = hoomd.md.nlist.Tree(0.3)
lj = pair.KA_LJ(tree)
nvt = hoomd.md.methods.NVT(hoomd.filter.All(), 1.0, 0.25)
integrator.forces = [lj]
integrator.methods = [nvt]

sim.operations.integrator = integrator

sim.run(0)

In [39]:
trigger = hoomd.trigger.Periodic(100)
remove_drift = hoomd.update.CustomUpdater(trigger, CustomAction())
sim.operations.updaters.clear()
sim.operations.updaters.append(remove_drift)

In [40]:
remove_drift.act(sim.timestep)




2000
[-1.48723362  3.09052739  0.        ]
1
59


In [31]:
sim.run(1000)




1000
[[-3.63592088  1.21217563  0.        ]
 [-3.61497589  2.31349965  0.        ]
 [-3.5867572  -1.86618162  0.        ]
 [-3.5288457  -3.5737495   0.        ]
 [-3.38929813 -0.76031246  0.        ]
 [-3.25130896  0.34242841  0.        ]
 [-3.12679116 -2.73431658  0.        ]
 [-2.95516298  2.88780907  0.        ]
 [-2.94285819  1.63979223  0.        ]
 [-2.75667968 -3.48536957  0.        ]]
[ 3  9 20 29 38 45 47 62 63  6]
[48 56 24  0 32 40  9 49 41  1]



1100
[[ 3.5902266   0.95346076  0.        ]
 [-3.54930212  2.23302674  0.        ]
 [ 3.64684708 -1.83178577  0.        ]
 [-3.42631106 -3.43174716  0.        ]
 [-3.31805404 -0.89478568  0.        ]
 [-3.18536828  0.09539848  0.        ]
 [-3.10595973 -2.4351583   0.        ]
 [-2.98806146  2.92007058  0.        ]
 [-3.11488002  1.5179515   0.        ]
 [-2.62874357 -3.42045343  0.        ]]
[ 3  9 20 29 38 45 47 62 63  6]
[48 56 24  0 32 40  9 49 41  1]



1200
[[-3.57756346 -1.82182734  0.        ]
 [-3.5725834  -3.48475181  

In [43]:
sim.state.get_snapshot().particles.position[59]

array([-1.48723362,  3.09052739,  0.        ])

In [47]:
sim.state.get_snapshot()._cpp_obj.particles.position[59]

array([-1.48723362,  3.09052739,  0.        ])